In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
path='../input/spaceship-titanic'

In [ ]:
train_df=pd.read_csv(path+'/train.csv')
test_df=pd.read_csv(path+'/test.csv')

In [ ]:
train_df

In [ ]:
train_df.describe()

check null values per column and total null values

In [ ]:
print(train_df.isnull().sum())
print("Total NUll Values:",train_df.isnull().sum().sum())

check null values in luxury amenities while cryosleep is true because when cryosleep is true luxury amenities should be zero

In [ ]:
print("CryoSleep true and RoomService NUll: ",train_df.query('CryoSleep==True').RoomService.isnull().sum())
print("CryoSleep true and FoodCourt NUll: ",train_df.query('CryoSleep==True').FoodCourt.isnull().sum())
print("CryoSleep true and ShoppingMall NUll: ",train_df.query('CryoSleep==True').ShoppingMall.isnull().sum())
print("CryoSleep true and Spa NUll: ",train_df.query('CryoSleep==True').Spa.isnull().sum())
print("CryoSleep true and VRDeck NUll: ",train_df.query('CryoSleep==True').VRDeck.isnull().sum())

As when CryoSleep is true then RoomService,FoodCourt,ShoppongMall,Spa and VRDeck should be zero but there are NULL values so we will change it to Zero

In [ ]:
train_df.loc[train_df.CryoSleep==True,['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]=0

In [ ]:
print(train_df.isnull().sum())
print("Total NUll Values:",train_df.isnull().sum().sum())

As CryoSleep has NUll values so by comparing it with luxury amenties like RoomService,FoodCourt,ShoppingMall,Spa,VRDeck when they are zero it would indicate that they in CryoSleep I-e CryoSleep would be true for them, so we will make it true from NULL

In [ ]:
train_df.query('RoomService ==0 & FoodCourt==0 & ShoppingMall==0 & Spa==0& VRDeck==0& CryoSleep!=False &CryoSleep!=True')

In [ ]:
train_df.loc[train_df.query('RoomService ==0 & FoodCourt==0 & ShoppingMall==0 & Spa==0& VRDeck==0& CryoSleep!=False &CryoSleep!=True')
                            .index,'CryoSleep']=True

And for remaining CryoSleep when luxury amenties like RoomService,FoodCourt,ShoppingMall,Spa,VRDeck when **either of them is greater than zero** it would indicate that they in **not in CryoSleep** I-e CryoSleep would be False for them so we will make it **False** from NULL

In [ ]:
train_df.query('CryoSleep!=False &CryoSleep!=True &(RoomService>0 | FoodCourt>0 | ShoppingMall>0 | Spa>0| VRDeck>0)')

In [ ]:
train_df.loc[train_df.query('(RoomService>0 | FoodCourt>0 | ShoppingMall>0 | Spa>0| VRDeck>0)& CryoSleep!=False &CryoSleep!=True')
                            .index,'CryoSleep']=False

check Cryosleep remaining NULL values

In [ ]:
train_df.query('CryoSleep!=False &CryoSleep!=True')

now by checking CryoSleep there are 11 remaining null Values and w.r.t luxury amenities like RoomService,FoodCourt,ShoppingMall,Spa,VRDeck **either of them is NULL while others are zero** so by logic we will go with **making null luxury amenity zero** as other four luxury amenity are zero and then **make Cryosleep True** as all luxury amenities will be zero

making NUll luxury amenities equal to zero while CryoSleep is NULL

In [ ]:
train_df.loc[train_df.query('CryoSleep!=False &CryoSleep!=True')
                            .index,['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']]=0

Make remaining Cryosleep equal to True as luxury amenities is zero

In [ ]:
train_df.loc[train_df.query('CryoSleep!=False &CryoSleep!=True')
                            .index,'CryoSleep']=True

In [ ]:
print(train_df.isnull().sum())
print("Total NUll Values:",train_df.isnull().sum().sum())

check median of numeric variables

In [ ]:
train_df.median(numeric_only=True)

now we will fill the null values of numeric variables, Age,RoomService,Spa,FoodCourt,ShoppingMall,VRDeck by its median

In [ ]:
train_df.fillna(train_df.median(numeric_only=True),inplace=True)

In [ ]:
print(train_df.isnull().sum())
print("Total NUll Values:",train_df.isnull().sum().sum())

now we will add spending column which total of luxury amenities

In [ ]:
train_df['spending']=train_df['RoomService']+train_df['FoodCourt']+train_df['ShoppingMall']+train_df['Spa']+train_df['VRDeck']

In [ ]:
train_df.query('spending>0')['spending'].mean()

checking vip null values w.r.t HomePlanet **Earth** and spending

In [ ]:
train_df.query('VIP!=True & VIP!=False & `HomePlanet` =="Earth" & spending<2500 & spending>0')

By checking various configuration i saw there are total 115 VIP null values for HomePlanet **Earth** and among known VIP values there is **zero** VIP true for **Earth**  and among 115, 49 have 0 spending so we can make VIP False for them and among remaining 66, 64 are less than spending mean so we will make VIP False for all null where HomePlanet is Earth

In [ ]:
train_df.loc[train_df.query('VIP!=True & VIP!=False & `HomePlanet` =="Earth"')
                            .index,'VIP']=False

checking vip null values w.r.t HomePlanet **Europa** and spending


In [ ]:
train_df.query('VIP!=True & VIP!=False & `HomePlanet` =="Europa"& spending>0 & spending<2500')

there are 42 VIP null values for HomePlanet Europa and among them 5 have spending value less than spending mean(2500) and are closer to 2000 so we will make VIP true for all when HomePlanet is **Europa**

In [ ]:
train_df.loc[train_df.query('VIP!=True & VIP!=False & `HomePlanet` =="Europa"')
                            .index,'VIP']=True

checking vip null values w.r.t HomePlanet **Mars** and spending

In [ ]:
train_df.query('VIP!=True & VIP!=False & `HomePlanet` =="Mars"& spending>0 & spending<2500')

there are 43 VIP null values for HomePlanet Mars and among them 24 have spending value **zero** so to make things simple we will make VIP null values False for when Spending is zero for Mars and remaining true

In [ ]:
train_df.loc[train_df.query('VIP!=True & VIP!=False & `HomePlanet` =="Mars"& spending==0')
                            .index,'VIP']=False

train_df.loc[train_df.query('VIP!=True & VIP!=False & `HomePlanet` =="Mars"& spending>0')
                            .index,'VIP']=True

Now there are 3 remaining VIP null as there Homeplanet is null so we will make them False as there spending is low than 2500

In [ ]:
train_df.loc[train_df.query('VIP!=True & VIP!=False')
                            .index,'VIP']=False

In [ ]:
print(train_df.isnull().sum())
print("Total NUll Values:",train_df.isnull().sum().sum())

now we have NULL values in categorical variables we have few options to dea with it, i will choose 2 methods I-e drop all NULL values and replace NULL values with new value. we will then compare which gives us good results

In [ ]:
train_drop_nan=train_df.dropna()

In [ ]:
print(train_drop_nan.isnull().sum())

giving dummy names to null values in different columns

In [ ]:
train_df.loc[train_df.query('`HomePlanet`!="Earth" &`HomePlanet`!="Europa" &`HomePlanet`!="Mars"')
                            .index,'HomePlanet']="Unknown"

train_df.loc[train_df[train_df.Cabin.isna()==True]
                            .index,'Cabin']="Not_sure"

train_df.loc[train_df[train_df.Destination.isna()==True]
                            .index,'Destination']="not_known"

train_df.loc[train_df[train_df.Name.isna()==True]
                            .index,'Name']="no_name"

In [ ]:
print(train_df.isnull().sum())
print("Total NUll Values:",train_df.isnull().sum().sum())